# Tick data

For optimum results this notebook should be run during the Forex trading session.

In [25]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('151.80.45.44', 7496, clientId=15)

<IB connected to 151.80.45.44:7496 clientId=15>

### Streaming tick data

Create some Forex contracts:

In [26]:
contracts = [Forex(pair) for pair in 'USDJPY GBPUSD USDCHF USDCAD AUDUSD'.split()]

eurusd = contracts[0]

Request streaming ticks for them:

In [27]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)

Wait a few seconds for the tickers to get filled.

In [28]:
ticker = ib.ticker(eurusd)
ib.sleep(2)
print(eurusd)

ticker

Forex('USDJPY', exchange='IDEALPRO')


Ticker(contract=Forex('USDJPY', exchange='IDEALPRO'), time=datetime.datetime(2018, 11, 30, 21, 45, 21, 231789, tzinfo=datetime.timezone.utc), bid=113.535, bidSize=1000000, ask=113.545, askSize=14000000, prevBidSize=1035000, prevAskSize=15000000, high=113.715, low=113.34, close=113.48, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[])

The price of Forex ticks is always nan. To get a midpoint price use ``marketPrice()``.

The tickers are kept
live updated, try this a few times to see if the price changes:

In [29]:
ticker.marketPrice()

113.53999999999999

The following cell will start a 30 second loop that prints a live updated ticker table.
It is updated on every ticker change.

In [31]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(columns='symbol bidSize bid ask askSize high low close'.split())
df['symbol'] = [c.symbol + c.currency for c in contracts]
contract2Row = {c: i for (i, c) in enumerate(contracts)}

def onPendingTickers(tickers):
    for t in tickers:
        row = contract2Row[t.contract]
        df.iloc[row, 1:] = (t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
        clear_output(wait=True)
    display(df)        

ib.pendingTickersEvent += onPendingTickers
ib.sleep(30)
ib.pendingTickersEvent -= onPendingTickers

ValueError: Contract Forex('USDJPY', exchange='IDEALPRO') can't be hashed

New tick data is available in the 'ticks' attribute of the pending tickers.
The tick data will be cleared before the next update.

To stop the live tick subscriptions:

In [32]:
for contract in contracts:
    ib.cancelMktData(contract)

### Historical tick data

Historical tick data can be fetched with a maximum of 1000 ticks at a time. Either the start time or the end time must be given, and one of them must remain empty:

In [33]:
import datetime

start = ''
end = datetime.datetime.now()
ticks = ib.reqHistoricalTicks(eurusd, start, end, 1000, 'BID_ASK', useRth=False)

ticks[-1]

HistoricalTickBidAsk(time=datetime.datetime(2018, 11, 30, 21, 46, 34, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(), priceBid=113.53, priceAsk=113.535, sizeBid=1000000, sizeAsk=2000000)

In [34]:
ib.disconnect()